In [ ]:
!pip3 install seaborn

In [ ]:
#İmporting libaries

In [ ]:
!pip3 install statsmodels

In [ ]:
!pip3 install xgboost

In [ ]:
!pip3 install sklearn

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm
import xgboost

from sklearn.metrics import mean_squared_error,r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.ensemble import AdaBoostRegressor
from xgboost.sklearn import XGBRegressor 
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn import neighbors
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

In [ ]:
data=pd.read_csv("../input/marrket-mix-dataset/mktmix.csv")

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Reading the data

In [ ]:
data.head()

In [ ]:
data = data.rename(columns = { 'NewVolSales' : 'NewVolSales', 
                  'Base_Price' : 'Base_Price', 
                  'Radio ':  'Radio', 
                  'InStore' : 'InStore', 
                  'NewspaperInserts' : 'NewspaperInserts', 
                  'Discount' : 'Discount' , 
                  'TV' : 'TV', 
                  'Stout' : 'Stout', 
                  'Website_Campaign ':  'Web'})

In [ ]:
# Understanding the data

In [ ]:
data.info()

In [ ]:
data.shape

In [ ]:
data.isnull().sum()

In [ ]:
# Preprocessing

In [ ]:
data.head()

In [ ]:
data.isnull().sum()

In [ ]:
data["Web"] = data["Web"].fillna("other")
data["NewspaperInserts"] = data["NewspaperInserts"].fillna("other")
data["Radio"] = data["Radio"].fillna(data["Radio"].median())

In [ ]:
data.isnull().sum()

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
#Visulations

In [ ]:
plt.figure(figsize=(6,6))
sns.scatterplot(x=data['Base_Price'],y=data['NewVolSales'],hue=data['NewVolSales'])
plt.xlabel('Base_Price')
plt.ylabel('NewVolSales')
plt.show()

In [ ]:
plt.figure(figsize=(6,6))
sns.scatterplot(x=data['Radio'],y=data['NewVolSales'],hue=data['NewVolSales'])
plt.xlabel('Radio')
plt.ylabel('NewVolSales')
plt.show()

In [ ]:
plt.figure(figsize=(6,6))
sns.scatterplot(x=data['Discount'],y=data['NewVolSales'],hue=data['NewVolSales'])
plt.xlabel('Discount')
plt.ylabel('NewVolSales')
plt.show()

In [ ]:
corr_matrix = data.corr()
plt.figure(figsize=(5,5))
sns.clustermap(corr_matrix, annot = True, fmt = ".2f")
plt.title("Correlation between columns")
plt.show

In [ ]:
# Editing categorical variables

In [ ]:
cat_col = ["Web","NewspaperInserts"]
data = pd.get_dummies(data, columns = cat_col)

In [ ]:
data = data.drop(["Web_other","NewspaperInserts_other"], axis = 1)

In [ ]:
data.info()

In [ ]:
data.head()

In [ ]:
# Handling outliers 

In [ ]:
chechk_outliers = ["Base_Price","Radio","InStore","Discount","TV","Stout"]

In [ ]:
for c in chechk_outliers:
    plt.figure()
    sns.boxplot(x = c, data = data, orient = "v")

In [ ]:
# #

In [ ]:
outliers = ["Radio","Base_Price"]

In [ ]:
def treat_outlier(x):
    q5 = np.percentile(x,5)
    q25= np.percentile(x,25)
    q75= np.percentile(x,75)
    up_trend=np.percentile(x,95)
    IQR = q75-q25
    low_level = q25-(1.5*IQR)
    up_level = q75+(1.5*IQR)
    
    return x.apply(lambda y: up_trend if y> up_level else y).apply(lambda y: q5 if y < low_level else y)

In [ ]:
for i in data[outliers]:
    data[i]=treat_outlier(data[i])

In [ ]:
# Modelling 

In [ ]:
x = data.drop(["NewVolSales"],axis = 1)
y = data.NewVolSales

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.20, random_state = 0)

In [ ]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
x_train = pd.DataFrame(x_train)
x_test = pd.DataFrame(x_test)
x_train.columns = x.columns
x_test.columns = x.columns

In [ ]:
svr = SVR(kernel = "rbf")
ada = AdaBoostRegressor()
xgb = XGBRegressor()
rfr = RandomForestRegressor()
dtr = DecisionTreeRegressor()
gbr = GradientBoostingRegressor()
knn = neighbors.KNeighborsRegressor()
mlp = MLPRegressor()
names = ["SVR","AdaBoostRegressor","XGBRegressor","RandomForestRegressor","DecisionTreeRegressor",
        "GradientBoostingRegressor","KNeighborsRegressor","MLPRegressor"]
models = [svr, ada, xgb, rfr, dtr, gbr, knn, mlp]

In [ ]:
for name, clf in zip(names, models):
        clf.fit(x_train, y_train)
        y_pred = clf.predict(x_test)
        mse = mean_squared_error(y_test, y_pred) 
        print("{}: mse score: {}".format(name, mse))
        r2 = r2_score(y_test, y_pred)
        print("{}: r2 score: {}".format(name, r2))
        mae = mean_absolute_error(y_test, y_pred)
        print("{}: mea score: {}".format(name, mae))
        mape = np.mean(abs((y_pred-y_test)/y_test))*100
        print("{}: mape :{}".format(name,mape))
        print("\n")

In [ ]:
poly_reg = PolynomialFeatures(degree=2)
X_poly = poly_reg.fit_transform(x_train)

In [ ]:
lin_reg = LinearRegression()
lin_reg = lin_reg.fit(X_poly, y_train)

In [ ]:
y_pred = lin_reg.predict(poly_reg.transform(x_test))

In [ ]:
mse = mean_squared_error(y_test, y_pred)
print("MSE score",mse)
r2 = r2_score(y_test, y_pred)
print("r2 score",r2)
mea = mean_absolute_error(y_test, y_pred)
print("MAE score",mea)

In [ ]:
parametresGrid_rfr = {"max_depth":[1,3,5,8,10],
                 "max_features":[2,5,10],
                 "min_samples_split":[5,10,20],
                 "n_estimators":[50,100,150,200]}

In [ ]:
rfrr = GridSearchCV(rfr, parametresGrid_rfr, cv = 10, scoring="neg_mean_squared_error", refit = True,
                  n_jobs = -1, verbose = 2)

In [ ]:
rfrr.fit(x_train, y_train)

In [ ]:
rfrr.best_estimator_

In [ ]:
y_pred = rfrr.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
print("Random Forest Mse score : ", mse)
r2 = r2_score(y_test, y_pred)
print("Random Forest R2 score",r2)
mape = np.mean(abs((y_pred-y_test)/y_test))*100
print("Random Forest mape score",mape)

In [ ]:
parametresGrid_gbr = {"max_depth":[3,5,8,10],
                 "max_features":[2,5,10],
                 "min_samples_split":[10,50,100],
                 "n_estimators":[100,200,500,1000]}

In [ ]:
gbrr = GridSearchCV(gbr, parametresGrid_gbr, cv = 10, scoring="neg_mean_squared_error", refit = True,
                  n_jobs = -1, verbose = 2)

In [ ]:
gbrr.fit(x_train, y_train)

In [ ]:
gbrr.best_estimator_

In [ ]:
y_pred = gbrr.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
print("Gradient Boost Mse score : ", mse)
r2 = r2_score(y_test, y_pred)
print("Gradient Boost R2 score",r2)
mape = np.mean(abs((y_pred-y_test)/y_test))*100
print("Gradient Boost mape score",mape)

In [ ]:
parametresGrid_xgb = {"nthread":[4],"objective":["reg:linear"],
                 "learnin_rate":[0.03,0.05,0.07], "max_depth" : [5,6,7],
                 "min_child_weight":[4,5,6], "silent":[1],
                 "subsample":[0.7], "colsample_bytree":[0.7],
                 "n_estimators":[100,500,1000]}

In [ ]:
xgbr = GridSearchCV(xgb, parametresGrid_xgb, cv = 10, scoring="neg_mean_squared_error", refit = True,
                n_jobs = -1, verbose = 2)

In [ ]:
xgbr.fit(x_train, y_train)

In [ ]:
xgbr.best_estimator_

In [ ]:
xgbr = XGBRegressor(colsample_bytree=0.7, learnin_rate=0.03, max_depth=7,
             min_child_weight=4, nthread=4, silent=1, subsample=0.7).fit(x_train, y_train)

In [ ]:
y_pred = xgbr.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
print("XGBoost Mse score : ", mse)
r2 = r2_score(y_test, y_pred)
print("XGBoost R2 score",r2)
mape = np.mean(abs((y_pred-y_test)/y_test))*100
print("XGBoost mape score",mape)

In [ ]:
feat_importances = pd.Series(xgbr.feature_importances_, index=x.columns)
feat_importances.nlargest(10).plot(kind='barh',figsize=(7,7))

In [ ]:
xgbr.feature_importances_

In [ ]:
y_pred = xgbr.predict(x_train)

In [ ]:
model_xgb = sm.ols(formula="y_pred~x_train", data=x_train).fit()
print(model_xgb.summary())

In [ ]:
 def get_params(x,y):
        model = sm.ols(formula="y~x", data=x).fit()
        coef = model.params
        coef = coef[1:]
        return coef

In [ ]:
coef = get_params(x_train,y_pred)
coef

In [ ]:
def elasticty(x,y,get_params):
      x_mean = x.mean()
      elasticity = np.column_stack([coef, x_mean]).tolist()
      elasticity = pd.DataFrame(elasticity)
      elasticity.columns = (["Coeffiecents","Mean"])
      elasticity["elasticty_value"] = elasticity["Coeffiecents"]*(elasticity["Mean"]/y.mean())
      elasticity.head(20)
      return elasticity

In [ ]:
elasticty_data = elasticty(x,y,coef)
elasticty_data

In [ ]:
sns.barplot(x=elasticty_data["elasticty_value"],
            y=x.columns, data=elasticty_data)
plt.show()